In [2]:
import pandas as pd
import numpy as np
from rich.console import Console
from rich.progress import track
import os

BASE_PATH = "C:\\Users\\Haroldo Duraes\\Desktop\\GOvGO\\v0\\#DATA\\PNCP\\"
CAT_PATH = BASE_PATH + "CAT\\"
FILE = CAT_PATH + "Lista CATMAT.xlsx"
ABA = "Materiais"

# Configurar o console rich para uma melhor visualização
console = Console()

# Carregar os dados
console.print("[bold yellow]Carregando dados CATMAT...[/bold yellow]")
df = pd.read_excel(FILE, sheet_name=ABA)

# Verificar as dimensões e colunas
console.print(f"[green]DataFrame carregado: {df.shape[0]} linhas, {df.shape[1]} colunas[/green]")
console.print("Primeiras linhas:")
display(df.head())

Carregando dados CATMAT...

DataFrame carregado: 157853 linhas, 9 colunas

Primeiras linhas:

,Código do Grupo,Nome do Grupo,Código da Classe,Nome da Classe,Código do PDM,Nome do PDM,Código do Item,Descrição do Item,Código NCM
0,10,ARMAMENTO,1005,ARMAS DE FOGO DE CALIBRE ATÉ 120MM,1712,PEÇAS / ACESSÓRIOS ARMAMENTO,446820,"PEÇAS / ACESSÓRIOS ARMAMENTO, MATERIAL:AÇO, TI...",-
1,10,ARMAMENTO,1005,ARMAS DE FOGO DE CALIBRE ATÉ 120MM,1712,PEÇAS / ACESSÓRIOS ARMAMENTO,446821,"PEÇAS / ACESSÓRIOS ARMAMENTO, MATERIAL:AÇO, TI...",-
2,10,ARMAMENTO,1005,ARMAS DE FOGO DE CALIBRE ATÉ 120MM,1712,PEÇAS / ACESSÓRIOS ARMAMENTO,446823,"PEÇAS / ACESSÓRIOS ARMAMENTO, MATERIAL:AÇO, TI...",-
3,10,ARMAMENTO,1005,ARMAS DE FOGO DE CALIBRE ATÉ 120MM,1712,PEÇAS / ACESSÓRIOS ARMAMENTO,446840,"PEÇAS / ACESSÓRIOS ARMAMENTO, MATERIAL:AÇO, TI...",-
4,10,ARMAMENTO,1005,ARMAS DE FOGO DE CALIBRE ATÉ 120MM,1712,PEÇAS / ACESSÓRIOS ARMAMENTO,446842,"PEÇAS / ACESSÓRIOS ARMAMENTO, MATERIAL:AÇO, TI...",-


In [3]:

# Criar o DataFrame simplificado com apenas as colunas necessárias
console.print("\n[bold yellow]Criando DataFrame simplificado...[/bold yellow]")

catmat_df = pd.DataFrame()
catmat_df['ITEM'] = df['Descrição do Item']
catmat_df['GRUPO'] = df['Código do Grupo'].astype(str) + ' - ' + df['Nome do Grupo']

console.print("[green]DataFrame simplificado criado[/green]")
console.print("Exemplo de dados:")
display(catmat_df.head())


Criando DataFrame simplificado...

DataFrame simplificado criado

Exemplo de dados:

,ITEM,GRUPO
0,"PEÇAS / ACESSÓRIOS ARMAMENTO, MATERIAL:AÇO, TI...",10 - ARMAMENTO
1,"PEÇAS / ACESSÓRIOS ARMAMENTO, MATERIAL:AÇO, TI...",10 - ARMAMENTO
2,"PEÇAS / ACESSÓRIOS ARMAMENTO, MATERIAL:AÇO, TI...",10 - ARMAMENTO
3,"PEÇAS / ACESSÓRIOS ARMAMENTO, MATERIAL:AÇO, TI...",10 - ARMAMENTO
4,"PEÇAS / ACESSÓRIOS ARMAMENTO, MATERIAL:AÇO, TI...",10 - ARMAMENTO


In [5]:
# Fazer amostragem dos dados - no máximo n itens por GRUPO e divisão na proporção P:1
console.print("\n[bold yellow]Criando amostra representativa e divisão treino/validação...[/bold yellow]")

n = 200  # Máximo de itens por grupo
p = 3    # Proporção para treinamento:validação (4:1)

# Preparar os DataFrames finais
training_df = pd.DataFrame(columns=['ITEM', 'CATEGORIA'])
validation_df = pd.DataFrame(columns=['ITEM', 'CATEGORIA'])

# Função para processar cada grupo
def sample_and_split_group(group_name, group_data):
    global training_df, validation_df
    
    # Se o grupo tem mais itens que n, fazer amostragem
    if len(group_data) > n:
        sampled_group = group_data.sample(n=n, random_state=42)
    else:
        sampled_group = group_data
    
    # Calcular o número de itens para treinamento e validação
    total_items = len(sampled_group)
    train_size = int(total_items * p / (p + 1))
    # O restante vai para validação
    
    # Embaralhar os dados antes de dividir
    shuffled_group = sampled_group.sample(frac=1, random_state=42)
    
    # Dividir em treinamento e validação
    train_items = shuffled_group.iloc[:train_size]
    val_items = shuffled_group.iloc[train_size:]
    
    # Adicionar aos DataFrames com a categoria formatada
    category = "MATERIAL; " + group_name
    
    # Adicionar ao treinamento
    train_data = pd.DataFrame({
        'ITEM': train_items['ITEM'].tolist(),
        'CATEGORIA': [category] * len(train_items)
    })
    training_df = pd.concat([training_df, train_data], ignore_index=True)
    
    # Adicionar à validação
    val_data = pd.DataFrame({
        'ITEM': val_items['ITEM'].tolist(),
        'CATEGORIA': [category] * len(val_items)
    })
    validation_df = pd.concat([validation_df, val_data], ignore_index=True)
    
    return sampled_group  # Retorna o grupo amostrado para o sample_df

# Criar sample_df para armazenar todos os itens amostrados
sample_df = pd.DataFrame(columns=catmat_df.columns)

# Processar cada grupo
for group_name, group_data in track(catmat_df.groupby('GRUPO'), description="Processando grupos"):
    sampled_group = sample_and_split_group(group_name, group_data)
    sample_df = pd.concat([sample_df, sampled_group], ignore_index=True)

# Estatísticas sobre os conjuntos resultantes
console.print(f"\n[green]DataFrame original: {catmat_df.shape[0]} itens[/green]")
console.print(f"[green]Amostra total: {sample_df.shape[0]} itens ({sample_df.shape[0]/catmat_df.shape[0]*100:.2f}% do original)[/green]")
console.print(f"[green]Conjunto de treinamento: {training_df.shape[0]} itens ({training_df.shape[0]/sample_df.shape[0]*100:.2f}% da amostra)[/green]")
console.print(f"[green]Conjunto de validação: {validation_df.shape[0]} itens ({validation_df.shape[0]/sample_df.shape[0]*100:.2f}% da amostra)[/green]")
console.print(f"[green]Proporção treinamento:validação = {training_df.shape[0]/validation_df.shape[0]:.2f}:1[/green]")

# Número de grupos
num_grupos = training_df['CATEGORIA'].nunique()
console.print(f"[green]Número total de grupos: {num_grupos}[/green]")

# Exemplos
console.print("\nExemplos do conjunto de treinamento:")
display(training_df.head())

console.print("\nExemplos do conjunto de validação:")
display(validation_df.head())

# Distribuição de grupos
console.print("\nDistribuição de itens por grupo (top 10):")
grupo_counts = pd.DataFrame({
    'Treinamento': training_df['CATEGORIA'].value_counts(),
    'Validação': validation_df['CATEGORIA'].value_counts()
}).fillna(0).astype(int)

display(grupo_counts.head(10))

Criando amostra representativa e divisão treino/validação...

Output()

DataFrame original: 157853 itens

Amostra total: 12819 itens (8.12% do original)

Conjunto de treinamento: 9607 itens (74.94% da amostra)

...

,ITEM,CATEGORIA
0,"PEÇAS / ACESSÓRIOS ARMAMENTO, MATERIAL:AÇO, AP...",MATERIAL; 10 - ARMAMENTO
1,"PARAFUSO ARMAMENTO, TIPO:LIMITADOR, MATERIAL:A...",MATERIAL; 10 - ARMAMENTO
2,"PEÇAS / ACESSÓRIOS ARMAMENTO, APLICAÇÃO: VBTP ...",MATERIAL; 10 - ARMAMENTO
3,"PEÇAS / ACESSÓRIOS ARMAMENTO, MATERIAL:AÇO, TI...",MATERIAL; 10 - ARMAMENTO
4,"PEÇAS / ACESSÓRIOS ARMAMENTO, MATERIAL:METAL, ...",MATERIAL; 10 - ARMAMENTO


Exemplos do conjunto de validação:

,ITEM,CATEGORIA
0,"PEÇAS / ACESSÓRIOS ARMAMENTO, TIPO 3:BIPÉ, MAT...",MATERIAL; 10 - ARMAMENTO
1,"PEÇAS / ACESSÓRIOS ARMAMENTO, MATERIAL:AÇO, AP...",MATERIAL; 10 - ARMAMENTO
2,"PEÇAS / ACESSÓRIOS ARMAMENTO, MATERIAL:METAL, ...",MATERIAL; 10 - ARMAMENTO
3,"PEÇAS / ACESSÓRIOS ARMAMENTO, TIPO 4:PORCA DE ...",MATERIAL; 10 - ARMAMENTO
4,"PEÇAS / ACESSÓRIOS ARMAMENTO, TIPO 3:PARAFUSO ...",MATERIAL; 10 - ARMAMENTO


Distribuição de itens por grupo (top 10):

,Treinamento,Validação
CATEGORIA,,
MATERIAL; 10 - ARMAMENTO,150,50
MATERIAL; 11 - MATERIAIS BÉLICOS NUCLEARES,3,1
MATERIAL; 13 - MUNIÇÕES E EXPLOSIVOS,30,10
MATERIAL; 15 - AERONAVES E SEUS COMPONENTES ESTRUTURAIS,24,8
MATERIAL; 16 - COMPONENTES E ACESSORIOS DE AERONAVES,1,1
"MATERIAL; 19 - NAVIOS, PEQUENAS EMBARCACOES, PONTOES E DIQUES FLUTUANTES",58,20
MATERIAL; 20 - EQUIPAMENTOS PARA NAVIOS E EMBARCACOES,15,5
MATERIAL; 22 - EQUIPAMENTOS FERROVIÁRIOS,16,6
MATERIAL; 23 - VEíCULOS,150,50


In [6]:
# Definir o caminho para salvar o arquivo Excel
OUT_FILE = CAT_PATH + "catmat_datasets_nv1.xlsx"

# Criar um ExcelWriter para salvar múltiplas abas em um único arquivo Excel
with pd.ExcelWriter(OUT_FILE) as writer:
    # Salvar cada DataFrame em uma aba separada
    training_df.to_excel(writer, sheet_name="training", index=False)
    validation_df.to_excel(writer, sheet_name="validation", index=False)
    sample_df.to_excel(writer, sheet_name="sample", index=False)
    


print(f"Datasets salvos em: {OUT_FILE}")
print("Abas salvas:")
print("- training: conjunto de treinamento com prefixo MATERIAL")
print("- validation: conjunto de validação com prefixo MATERIAL")
print("- sample: dataset reduzido por amostragem")

Datasets salvos em: C:\Users\Haroldo Duraes\Desktop\GOvGO\v0\#DATA\PNCP\CAT\catmat_datasets_nv1.xlsx
Abas salvas:
- training: conjunto de treinamento com prefixo MATERIAL
- validation: conjunto de validação com prefixo MATERIAL
- sample: dataset reduzido por amostragem
